# <center> Feature generation: Basic
   

In [10]:
%store -r __ipy
%store -r __da
%store -r __ml

In [11]:
__ipy

Helper ipython script loaded


In [12]:
__da

Basic Data Analysis tools was loaded


In [13]:
__ml

In [14]:
train_df = pd.read_pickle(root + 'train_df.pkl')
test_df = pd.read_pickle(root + 'test_df.pkl')
train_df = train_df.query('not (site_id == 0 & meter == 0 & timestamp <= "2016-05-20") and not (meter == 2 & building_id == 1099)')

In [ ]:
%%time

train_df['age'] = train_df['year_built'].max() - train_df['year_built'] + 1
test_df['age'] = test_df['year_built'].max() - test_df['year_built'] + 1

le = LabelEncoder()
train_df['primary_use'] = train_df['primary_use'].astype(str)
train_df['primary_use'] = le.fit_transform(train_df['primary_use']).astype(np.int8)

test_df['primary_use'] = test_df['primary_use'].astype(str)
test_df['primary_use'] = le.fit_transform(test_df['primary_use']).astype(np.int8)

train_df['floor_count'] = train_df['floor_count'].fillna(-999).astype(np.int16)
test_df['floor_count'] = test_df['floor_count'].fillna(-999).astype(np.int16)

train_df['year_built'] = train_df['year_built'].fillna(-999).astype(np.int16)
test_df['year_built'] = test_df['year_built'].fillna(-999).astype(np.int16)

train_df['age'] = train_df['age'].fillna(-999).astype(np.int16)
test_df['age'] = test_df['age'].fillna(-999).astype(np.int16)

train_df['cloud_coverage'] = train_df['cloud_coverage'].fillna(-999).astype(np.int16)
test_df['cloud_coverage'] = test_df['cloud_coverage'].fillna(-999).astype(np.int16) 

train_df['month_datetime'] = train_df['timestamp'].dt.month.astype(np.int8)
train_df['weekofyear_datetime'] = train_df['timestamp'].dt.weekofyear.astype(np.int8)
train_df['dayofyear_datetime'] = train_df['timestamp'].dt.dayofyear.astype(np.int16)
    
train_df['hour_datetime'] = train_df['timestamp'].dt.hour.astype(np.int8)  
train_df['day_week'] = train_df['timestamp'].dt.dayofweek.astype(np.int8)
train_df['day_month_datetime'] = train_df['timestamp'].dt.day.astype(np.int8)
train_df['week_month_datetime'] = train_df['timestamp'].dt.day/7
train_df['week_month_datetime'] = train_df['week_month_datetime'].apply(lambda x: math.ceil(x)).astype(np.int8)
    
test_df['month_datetime'] = test_df['timestamp'].dt.month.astype(np.int8)
test_df['weekofyear_datetime'] = test_df['timestamp'].dt.weekofyear.astype(np.int8)
test_df['dayofyear_datetime'] = test_df['timestamp'].dt.dayofyear.astype(np.int16)
    
test_df['hour_datetime'] = test_df['timestamp'].dt.hour.astype(np.int8)
test_df['day_week'] = test_df['timestamp'].dt.dayofweek.astype(np.int8)
test_df['day_month_datetime'] = test_df['timestamp'].dt.day.astype(np.int8)
test_df['week_month_datetime'] = test_df['timestamp'].dt.day/7
test_df['week_month_datetime'] = test_df['week_month_datetime'].apply(lambda x: math.ceil(x)).astype(np.int8)


train_df['weekday'] = np.uint8(df['timestamp'].dt.weekday)
test_df['weekday'] = np.uint8(df['timestamp'].dt.weekday)



In [43]:
import holidays

def set_holidays(train_df):
    site_0_holidays = holidays.CountryHoliday('US', prov=None, state='FL')
    site_1_holidays = holidays.CountryHoliday('UK')
    site_2_holidays = holidays.CountryHoliday('US', prov=None, state='AZ')
    site_3_holidays = holidays.CountryHoliday('US', prov=None, state='DC')
    site_4_holidays = holidays.CountryHoliday('US', prov=None, state='CA')
    site_5_holidays = holidays.CountryHoliday('UK')
    site_6_holidays = holidays.CountryHoliday('US')

    site_7_holidays = holidays.CountryHoliday('CA', prov='Ontario')
    site_8_holidays = holidays.CountryHoliday('US', prov=None, state='FL')
    site_9_holidays = holidays.CountryHoliday('US', prov=None, state='TX')

    site_10_holidays = holidays.CountryHoliday('US', prov=None, state='MT')
    site_11_holidays = holidays.CountryHoliday('CA', prov='Ontario')
    site_12_holidays = holidays.CountryHoliday('IE')

    site_13_holidays = holidays.CountryHoliday('US', state='MN')
    site_14_holidays = holidays.CountryHoliday('US', state='VA')
    site_15_holidays = holidays.CountryHoliday('US')


    holiday_dict = dict(list(zip(list(range(16)), [site_0_holidays,
                                site_1_holidays,
                                site_2_holidays, 
                                site_3_holidays ,
                                site_4_holidays ,
                                site_5_holidays ,
                                site_6_holidays ,
                                site_7_holidays ,
                                site_8_holidays ,
                                site_9_holidays ,
                                site_10_holidays,
                                site_11_holidays,
                                site_12_holidays,
                                site_13_holidays,
                                site_14_holidays,
                                site_15_holidays])))
    train_df['is_holiday'] = train_df[['timestamp', 'site_id']].apply(lambda x: x['timestamp'] in holiday_dict[x['site_id']], axis=1)
    return train_df

train_df = set_holidays(train_df)
test_df = set_holidays(test_df)

103    True
104    True
105    True
106    True
107    True
dtype: bool

## Generate full key for objects

In [ ]:
def to_key(train_df, test_df):
    le_key = LabelEncoder()
    train_df[KEY_NAME] = train_df[KEY].progress_apply(\
        lambda x: str(x['building_id']) + "_" + str(x['meter']) + "_" +   str(x['site_id']), axis=1)
    
    test_df[KEY_NAME] = test_df[KEY].progress_apply(\
        lambda x: str(x['building_id']) + "_" + str(x['meter']) + "_" +   str(x['site_id']), axis=1)
    
    le_key.fit(train_df[KEY_NAME])
    train_df[KEY_NAME] = le_key.transform(train_df[KEY_NAME])
    train_df[KEY_NAME] = train_df[KEY_NAME].astype(np.int32)
    
    test_df[KEY_NAME] = le_key.transform(test_df[KEY_NAME])
    test_df[KEY_NAME] = test_df[KEY_NAME].astype(np.int32)
    
    return train_df, test_df

train_df, test_df = to_key(train_df, test_df)

In [ ]:
def beafort_feature(df):
    beaufort_boundaries = [0, 0.3, 1.6, 3.4, 5.5, 8, 10.8, 13.9, 17.2, 20.8, 24.5, 28.5, 33, 10000]
    df['wind_speed_beaufort_scale'] = pd.cut(
        x=df['wind_speed'], 
        bins=beaufort_boundaries, 
        labels=range(len(beaufort_boundaries)-1),
        right=False,
    ).cat.codes
    
    df['wind_speed_beaufort_scale'] = np.uint8(df['wind_speed_beaufort_scale'])
    
    df['group'] = df['month']
    df['group'].replace((6, 7, 8), 21, inplace=True)
    df['group'].replace((9, 10, 11), 22, inplace=True)
    df['group'].replace((3, 4, 5), 23, inplace=True)
    df['group'].replace((1, 2, 12), 24, inplace=True)
    df['group'].replace((21), 1, inplace=True)
    df['group'].replace((22), 2, inplace=True)
    df['group'].replace((23), 3, inplace=True)
    df['group'].replace((24), 4, inplace=True)
    df['group'] = np.uint8(df['group'])
    return df

train_df = beafort_feature(train_df)
test_df = beafort_feature(test_df)

In [16]:
test_df.to_pickle(root + "test_features.pkl")
train_df.to_pickle(root + "train_features.pkl")